# Supervised Machine Learning - Oversampling Star_Galaxy

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import RandomOverSampler
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import extract

In [2]:
columns = ["obj_ID",'alpha','delta', 'u', 'g', 'r', 'i', 'z', 'run_ID',
       'rerun_ID', 'cam_col', 'field_ID', 'spec_obj_ID', 'redshift',
       'plate', 'MJD', 'fiber_ID']
target = ["class"]

In [3]:
# # Create engine
# engine = create_engine("sqlite:///

In [4]:
# # reflect an existing database into a new model
# Base = automap_base()
# # reflect the tables
# Base.prepare(engine, reflect=True)
# # Save references to each table
# Measurement = Base.classes.measurement
# Station = Base.classes.station

In [5]:
# # Create our session (link) from Python to the DB
# session = Session(engine)

In [6]:
# Load the data
file_path = Path('Resources/star_galaxy.csv', index=False)
data_df = pd.read_csv(file_path)
# df = df.loc[:, columns].copy()
data_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237660e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543780e+18,GALAXY,0.634794,5812,56354,171
1,1.237660e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176010e+19,GALAXY,0.779136,10445,58158,427
2,1.237660e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237660e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030110e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891860e+18,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81034,1.237680e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.060000e+19,GALAXY,0.000000,9374,57749,438
81035,1.237680e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586350e+18,GALAXY,0.404895,7626,56934,866
81036,1.237670e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112010e+18,GALAXY,0.143366,2764,54535,74
81037,1.237660e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,GALAXY,0.455040,6751,56368,470


In [7]:
data_df.set_index(['spec_obj_ID'], inplace = True)

In [8]:
data_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,class,redshift,plate,MJD,fiber_ID
spec_obj_ID,,,,,,,,,,,,,,,,,
6.543780e+18,1.237660e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,GALAXY,0.634794,5812,56354,171
1.176010e+19,1.237660e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,GALAXY,0.779136,10445,58158,427
5.152200e+18,1.237660e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,GALAXY,0.644195,4576,55592,299
1.030110e+19,1.237660e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,GALAXY,0.932346,9149,58039,775
6.891860e+18,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.060000e+19,1.237680e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,GALAXY,0.000000,9374,57749,438
8.586350e+18,1.237680e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,GALAXY,0.404895,7626,56934,866
3.112010e+18,1.237670e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,GALAXY,0.143366,2764,54535,74


In [9]:
 new_data_df=data_df.drop(['alpha','delta','run_ID','rerun_ID', 'cam_col', 'field_ID', "obj_ID",'plate', 'MJD', 'fiber_ID'], axis = 1 )

In [10]:
new_data_df["z"].min()

-9999.0

### Variable engineering

In [11]:
frequency = [1, 2, 3]

u_size_bins = [-10000, -6600,-3310, 33]
new_data_df["binned_u"] = pd.cut(new_data_df["u"], u_size_bins, labels = frequency)

g_size_bins = [-10000, -6600,-3310, 33]
new_data_df["binned_g"] = pd.cut(new_data_df["g"], g_size_bins, labels = frequency)

r_size_bins = [9,16,23, 30]
new_data_df["binned_r"] = pd.cut(new_data_df["r"], r_size_bins, labels = frequency)

i_size_bins = [9,17,25, 33]
new_data_df["binned_i"] = pd.cut(new_data_df["i"], i_size_bins, labels = frequency)

z_size_bins = [-10000, -6600,-3310, 30]
new_data_df["binned_z"] = pd.cut(new_data_df["z"], z_size_bins, labels = frequency)

new_data_df.sample(30)

,u,g,r,i,z,class,redshift,binned_u,binned_g,binned_r,binned_i,binned_z
spec_obj_ID,,,,,,,,,,,,
5.108210e+18,26.03472,22.99764,21.44613,20.26081,19.53234,GALAXY,0.623957,3,3,2,2,3
2.959040e+18,21.45943,20.12548,19.42720,18.98792,18.90500,STAR,-0.001003,3,3,2,2,3
7.978330e+18,24.48388,22.16685,20.53264,19.74641,19.21821,GALAXY,0.446152,3,3,2,2,3
2.429760e+18,22.99236,19.77799,18.07887,17.46940,17.03263,GALAXY,0.320553,3,3,2,2,3
5.858190e+18,22.44347,21.70276,20.07241,19.07501,18.57899,GALAXY,0.549382,3,3,2,2,3
1.297160e+18,21.32801,20.36130,20.04222,19.98708,20.30700,STAR,-0.000817,3,3,2,2,3
4.955660e+17,18.62236,16.90629,16.11058,15.70154,15.39523,GALAXY,0.053390,3,3,2,1,3
1.186720e+18,18.94766,16.88098,15.74930,15.28131,14.91052,GALAXY,0.150759,3,3,1,1,3
1.076930e+19,22.88588,22.44308,21.76550,21.21628,20.85812,GALAXY,0.838924,3,3,2,2,3


### Random Oversampling

In [12]:
# Create our features
X = new_data_df.drop('class', axis=1)

# Create our target
y = new_data_df['class']

X

,u,g,r,i,z,redshift,binned_u,binned_g,binned_r,binned_i,binned_z
spec_obj_ID,,,,,,,,,,,
6.543780e+18,23.87882,22.27530,20.39501,19.16573,18.79371,0.634794,3,3,2,2,3
1.176010e+19,24.77759,22.83188,22.58444,21.16812,21.61427,0.779136,3,3,2,2,3
5.152200e+18,25.26307,22.66389,20.60976,19.34857,18.94827,0.644195,3,3,2,2,3
1.030110e+19,22.13682,23.77656,21.61162,20.50454,19.25010,0.932346,3,3,2,2,3
6.891860e+18,19.43718,17.58028,16.49747,15.97711,15.54461,0.116123,3,3,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...
1.060000e+19,22.16759,22.97586,21.90404,21.30548,20.73569,0.000000,3,3,2,2,3
8.586350e+18,22.69118,22.38628,20.45003,19.75759,19.41526,0.404895,3,3,2,2,3
3.112010e+18,21.16916,19.26997,18.20428,17.69034,17.35221,0.143366,3,3,2,2,3


In [13]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'GALAXY': 44520, 'STAR': 16259})

In [15]:
# Resample the training data with the RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'GALAXY': 44520, 'STAR': 44520})

### Random Forest Classifier Algorithm (Stars and Galaxies)

In [16]:
# Train the Random Forest Classifier Model using the resampled data
model = RandomForestClassifier(n_estimators=100)
model.fit(X_resampled, y_resampled)

RandomForestClassifier()

In [17]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.998841034287338

In [18]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

     GALAXY       1.00      1.00      1.00      1.00      1.00      1.00     14925
       STAR       0.99      1.00      1.00      1.00      1.00      1.00      5335

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     20260



In [19]:
# Display the confusion matrix
c= confusion_matrix(y_test,y_pred)
cmros = pd.DataFrame(c, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cmros

,Predicted 0,Predicted 1
Actual 0,14896,29
Actual 1,2,5333


### Logestic Regression Algorithm (Stars and Galaxies)

In [20]:
model1 = LogisticRegression(solver='lbfgs', random_state=1)
model1.fit(X_resampled, y_resampled)

/Users/jane.mathew/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=1)

In [21]:
y_pred = model1.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9825803808169244

In [22]:
confusion_matrix(y_test, y_pred)

array([[14433,   492],
       [   10,  5325]])

In [23]:
print(classification_report_imbalanced(y_test, y_pred))


                   pre       rec       spe        f1       geo       iba       sup

     GALAXY       1.00      0.97      1.00      0.98      0.98      0.96     14925
       STAR       0.92      1.00      0.97      0.95      0.98      0.97      5335

avg / total       0.98      0.98      0.99      0.98      0.98      0.96     20260



In [ ]:
# Display the confusion matrix
c= confusion_matrix(y_test,y_pred)
cmlbgfs = pd.DataFrame(c, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cmlbgfs